In [2]:
from pymongo import MongoClient
# from sshtunnel import SSHTunnelForwarder
import pymongo
from tqdm import tqdm
import os
import re

import numpy as np
import pandas as pd

import dateutil.parser

In [3]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
! nvidia-smi

Sun May 29 19:25:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 35%   45C    P8    38W / 260W |     22MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 30%   

# MongoDB

In [5]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    # db_de = db.patent_de
    # db_us = db.patent_us
    # db_cn = db.patent_cn

    db_rm_us = db.rm_patent_us
    db_rm_cn = db.rm_patent_cn
    db_rm_de = db.rm_patent_de
    
    db_rm_us_2020 = db.rm_patent_us_2020
    db_rm_cn_2020 = db.rm_patent_cn_2020
    db_rm_de_2020 = db.rm_patent_de_2020

except errors.ConnectionFailure as err:
    print(err)

In [6]:
db_rm_us_2020

Collection(Database(MongoClient(host=['140.117.69.70:30241'], document_class=dict, tz_aware=False, connect=True, authmechanism='SCRAM-SHA-1'), 'eva'), 'rm_patent_us_2020')

# DE-CN-US

## 2020

### DE

In [7]:
data_assignee = db_rm_de_2020.aggregate( [ 
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}

 ] )


ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
# for data in data_assignee:
#     print(data)

In [8]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_rm_de_2020.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

,company,cpc,count
0,Intel Corp,G06F9,118
1,Intel Corp,G06N3,57
2,Intel Corp,G06F12,37
3,Intel Corp,G06T1,37
4,Intel Corp,G06T15,32


In [9]:
df_cpc.shape

(2063, 3)

In [10]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [11]:
# 轉成company-cpc 的總數表格
table_de = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_de = table_de.droplevel(level=0, axis=1)
table_de = table_de.rename(columns={'':'company'})

In [12]:
# 取有英文的部份
c_ls_de = [c for c in table_de.columns if re.search('[A-Za-z]', c)]
final_table_de = table_de[c_ls_de]
final_table_de.head()

cpc_n,company,A47,A61,A63,B06,B23,B60,B63,C01,C23,...,G10,G11,G16,H01,H02,H03,H04,H05,Y02,Y04
0,Agvolution I Gr GmbH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Apple Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,1.0,0.0
2,Attopsemi Technology Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Balluff GmbH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Barkhausen Inst Ggmbh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0


In [13]:
final_table_de.shape

(82, 32)

In [14]:
final_table_de.to_csv('rm_company_cpc_de_2020.csv',index=0)

把final_table資料存進MongoDB

In [35]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection_rm = db.rm_company_cpc_us_2020

In [36]:
# title
conn_db_collection_rm.insert_one({"company":final_table_us.columns.tolist()[0],"cpc":final_table_us.columns.tolist()[1:]})

# 內容
for k in final_table_us.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection_rm.insert_one({"company":k[0],"cpc":k[1:]})

### CN

In [17]:
data_assignee = db_rm_cn_2020.aggregate( [ 
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}

 ] )


ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
# for data in data_assignee:
#     print(data)

In [18]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_rm_cn_2020.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

,company,cpc,count
0,Huawei Technologies Co Ltd,H04W72,114
1,Huawei Technologies Co Ltd,00,107
2,Huawei Technologies Co Ltd,04,74
3,Huawei Technologies Co Ltd,08,73
4,Huawei Technologies Co Ltd,H04W76,68


In [19]:
df_cpc.shape

(32361, 3)

In [20]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [21]:
# 轉成company-cpc 的總數表格
table_cn = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_cn = table_cn.droplevel(level=0, axis=1)
table_cn = table_cn.rename(columns={'':'company'})

In [22]:
# 取有英文的部份
c_ls_cn = [c for c in table_cn.columns if re.search('[A-Za-z]', c)]
final_table_cn = table_cn[c_ls_cn]
final_table_cn.head()

cpc_n,company,A01,A23,A41,A42,A43,A44,A45,A47,A61,...,G10,G11,G16,H01,H02,H03,H04,H05,Y02,Y04
0,1230 Intelligent Technology Haian Co ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
1,AGC Glass Europe SA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0
2,AGC Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ARM Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9.0,95.0,0.0,3.0,0.0
4,Accenture Global Solutions Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
final_table_cn.shape

(2369, 92)

In [24]:
final_table_cn.to_csv('rm_company_cpc_cn_2020.csv',index=0)

把final_table資料存進MongoDB

In [25]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection_rm = db.rm_company_cpc_cn_2020

In [36]:
# title
conn_db_collection_rm.insert_one({"company":final_table_us.columns.tolist()[0],"cpc":final_table_us.columns.tolist()[1:]})

# 內容
for k in final_table_us.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection_rm.insert_one({"company":k[0],"cpc":k[1:]})

### US

In [27]:
data_assignee = db_rm_us_2020.aggregate( [ 
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}

 ] )


ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"].replace("))",")"))
    ls_count.append(data["count"])
    
# for data in data_assignee:
#     print(data)

In [28]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_rm_us_2020.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

,company,cpc,count
0,Qualcomm Inc,H04L5,4003
1,Qualcomm Inc,H04W72,3223
2,Qualcomm Inc,H04B7,1755
3,Qualcomm Inc,H04L1,1540
4,Qualcomm Inc,H04W52,921


In [29]:
df_cpc.shape

(54473, 3)

In [30]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [31]:
# 轉成company-cpc 的總數表格
table_us = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_us = table_us.droplevel(level=0, axis=1)
table_us = table_us.rename(columns={'':'company'})

In [32]:
# 取有英文的部份
c_ls_us = [c for c in table_us.columns if re.search('[A-Za-z]', c)]
final_table_us = table_us[c_ls_us]
final_table_us.head()

cpc_n,company,A01,A21,A23,A24,A41,A42,A44,A45,A47,...,G11,G16,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,#glowontech,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10353744 Canada Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
2,151 Research Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2524726 Ontario Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,8.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0
4,3M Innovative Properties Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0


In [33]:
final_table_us.shape

(1447, 88)

In [34]:
final_table_us.to_csv('rm_company_cpc_us_2020.csv',index=0)

把final_table資料存進MongoDB

In [35]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection_rm = db.rm_company_cpc_us_2020

In [36]:
# title
conn_db_collection_rm.insert_one({"company":final_table_us.columns.tolist()[0],"cpc":final_table_us.columns.tolist()[1:]})

# 內容
for k in final_table_us.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection_rm.insert_one({"company":k[0],"cpc":k[1:]})

In [37]:
final_table_de

cpc_n,company,A47,A61,A63,B06,B23,B60,B63,C01,C23,...,G10,G11,G16,H01,H02,H03,H04,H05,Y02,Y04
0,Agvolution I Gr GmbH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Apple Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,1.0,0.0
2,Attopsemi Technology Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Balluff GmbH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Barkhausen Inst Ggmbh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,Volkswagen AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0
78,Wago Verwaltungs GmbH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
79,Weber Hydraulik GmbH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
80,Wenko Wenselaar GmbH and Co KG,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 合併

In [41]:
# concat de cn us
df_c = pd.concat([final_table_de,final_table_cn], join='outer')
df_c2 = pd.concat([df_c,final_table_us], join='outer')
df_c2 = df_c2.fillna(0)
#reset index
t = list(df_c2.sort_index(axis=1).columns)
df_c2 = df_c2.reindex(columns=[t[-1]]+t[:-1])
df_c2.head()

cpc_n,company,A01,A21,A23,A24,A41,A42,A43,A44,A45,...,G11,G16,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,Agvolution I Gr GmbH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Apple Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,30.0,0.0,1.0,0.0,0.0
2,Attopsemi Technology Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Balluff GmbH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Barkhausen Inst Ggmbh,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0


In [42]:
# 合併重複的公司
df_c2 = df_c2.groupby(df_c2.company, axis=0).sum().reset_index()
df_c2.head()

cpc_n,company,A01,A21,A23,A24,A41,A42,A43,A44,A45,...,G11,G16,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,#glowontech,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10353744 Canada Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
2,1230 Intelligent Technology Haian Co ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
3,151 Research Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2524726 Ontario Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,8.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0


In [43]:
df_c2.to_csv('rm_company_cpc_all_2020.csv',index=0)

In [44]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.rm_company_cpc_all_2020

In [45]:
# title
conn_db_collection.insert_one({"company":df_c2.columns.tolist()[0],"cpc":df_c2.columns.tolist()[1:]})

# 內容
for k in df_c2.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"cpc":k[1:]})